<a href="https://colab.research.google.com/github/krish7101/100-days-of-code-python/blob/main/Recursive_Reprompting_and_Revision.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Source: https://arxiv.org/pdf/2210.06774.pdf

Get your openai api key from settings: https://platform.openai.com/account/api-keys

In [ ]:
# get the openai secret key
import getpass

secret_key = getpass.getpass('Please enter your openai key: ')

Please enter your openai key: ··········


In [ ]:
!pip install openai

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 75.5/75.5 kB 1.8 MB/s eta 0:00:00


In [ ]:
# get openai set up
import openai

openai.api_key = secret_key

def complete(prompt, temperature=1, max_tokens=500, stop=None):
    response = openai.ChatCompletion.create(
        model="gpt-4",
        messages=[
            {
            "role": "user",
            "content": prompt
            }
        ],
        temperature=temperature,
        max_tokens=max_tokens,
        top_p=1,
        frequency_penalty=0,
        presence_penalty=0,
        stop=stop
    )
    return response['choices'][0]['message']['content']

complete("is this working?")

"Yes, it's working. How can I assist you today?"

In [ ]:
# Add your premise
premise_input = "A new law grad returns home to start her career, but struggles with the broken justice system." #@param {type:"string"}
premise = f"Premise: {premise_input}"
premise

'Premise: A new law grad returns home to start her career, but struggles with the broken justice system.'

## Plan Module

In [ ]:
# Setting
setting_prompt = f"{premise}\nThe story is set in"
setting_response = complete(setting_prompt, stop=["."])
setting = f"Setting: The story is set in {setting_response}."
setting

'Setting: The story is set in a small, rural town in Kentucky called Blue Mountain.'

In [ ]:
# Characters
num_characters = 2
characters = ""
character_format = "Please invent characters for this story, in the format of [name] is [context]. For example 'Liza Turner is a 22-year-old woman' or 'Peyton Turner is Liza's older sister'. Each character is distinct and does not reuse the same name as these examples."
character_prompt = f"Premise: {premise}\n\nSetting: {setting}\n\n{character_format}\n\n"
for n in range(num_characters):
    character_prompt += f"{n+1}. Character Portrait:\n"
    character_response = complete(f"{character_prompt}\n\n{n+1}. Character Portrait:\n", stop=["."])
    character_prompt += f"{character_response}\n\n"
    characters += f"{n+1}. Character Portrait:\n{character_response.strip()}.\n\n"

print(characters)

1. Character Portrait:
Harper Davis is a determined 25-year-old recent law school graduate who is eager to make a difference, fiercely intelligent, and compassionate.

2. Character Portrait:
Donovan "Donny" Grant is a 40-year-old charismatic yet scheming town councilman who seems obsessed with maintaining the status quo.




In [ ]:
# Plot
plot_prompt = f"{premise}\n\n{setting}\n\n{characters}Outline the main plot points of the story.\n1."
plot_response = complete(plot_prompt)
plot = f"{plot_response}"
print("Outline the main plot points of the story\n\n1. ", plot)

Outline the main plot points of the story

1.  Harper Davis returns to her hometown after graduating law school, and quickly realizes that things have changed. The justice system is vaguely corrupt, twisted in favor of the wealthy and powerful, and the law she dedicated her years to studying seems non-existent.

2. Harper crosses paths with Donovan "Donny" Grant, the town councilman. His charming yet sly persona makes Harper suspicious about his role in this broken justice system.

3. Subsequently, Harper finds herself in a battle to unravel the corruption in the town’s justice system. She is eager to bring change and justice to the people of Blue Mountain but faces resistance from the town council, especially Donny, who uses his power and charisma to keep things the way they are.

4. Harper starts collecting evidence of corruption and helping those wronged by the system, increasingly clashing with Donny and others desperate to maintain the status quo.

5. After a major courtroom battl

In [ ]:
# Full Plan Module
print(f"{premise}\n\n{setting}\n\n{characters}\n\nOutline the main plot points of the story\n\n1. {plot}")

Premise: A new law grad returns home to start her career, but struggles with the broken justice system.

Setting: The story is set in a small, rural town in Kentucky called Blue Mountain.

1. Character Portrait:
Harper Davis is a determined 25-year-old recent law school graduate who is eager to make a difference, fiercely intelligent, and compassionate.

2. Character Portrait:
Donovan "Donny" Grant is a 40-year-old charismatic yet scheming town councilman who seems obsessed with maintaining the status quo.



Outline the main plot points of the story

1. Harper Davis returns to her hometown after graduating law school, and quickly realizes that things have changed. The justice system is vaguely corrupt, twisted in favor of the wealthy and powerful, and the law she dedicated her years to studying seems non-existent.

2. Harper crosses paths with Donovan "Donny" Grant, the town councilman. His charming yet sly persona makes Harper suspicious about his role in this broken justice system.


## Draft Module

In [ ]:
import re

plot_points_raw = plot.split('\n')
plot_points = []

for pp in plot_points_raw:
    if pp == '':
        continue
    else:
        stripped_pp = re.sub(r'^\d+\.\s*', '', pp)
        plot_points.append(stripped_pp)

print(plot_points)

['Harper Davis returns to her hometown after graduating law school, and quickly realizes that things have changed. The justice system is vaguely corrupt, twisted in favor of the wealthy and powerful, and the law she dedicated her years to studying seems non-existent.', 'Harper crosses paths with Donovan "Donny" Grant, the town councilman. His charming yet sly persona makes Harper suspicious about his role in this broken justice system.', 'Subsequently, Harper finds herself in a battle to unravel the corruption in the town’s justice system. She is eager to bring change and justice to the people of Blue Mountain but faces resistance from the town council, especially Donny, who uses his power and charisma to keep things the way they are.', 'Harper starts collecting evidence of corruption and helping those wronged by the system, increasingly clashing with Donny and others desperate to maintain the status quo.', 'After a major courtroom battle against Donny over a high-profile case, Harper 

In [ ]:
drafts = []

for idx in range(len(plot_points)):
    if idx == 0:
        previous = ""
        immediately = ""
    else:
        previous = f"Previous story summary:\n{plot_points[idx-1]}"
        immediately = f"Immediately before the current passage:\n{drafts[idx-1]}"

    upcoming = f"In the upcoming passage,\n{plot_points[idx]}"

    draft_prompt = f"Relevant context:\n{characters}{previous}\n\n{immediately}\n\n{upcoming}\n\nFull text below:"

    draft_response = complete(draft_prompt)
    draft_response = draft_response.strip()

    print(f"Plot Point {idx+1}\n-----\n", draft_prompt, f"\n{draft_response}\n-----\n\n")
    drafts.append(draft_response)

Plot Point 1
-----
 Relevant context:
1. Character Portrait:
Harper Davis is a determined 25-year-old recent law school graduate who is eager to make a difference, fiercely intelligent, and compassionate.

2. Character Portrait:
Donovan "Donny" Grant is a 40-year-old charismatic yet scheming town councilman who seems obsessed with maintaining the status quo.





In the upcoming passage,
Harper Davis returns to her hometown after graduating law school, and quickly realizes that things have changed. The justice system is vaguely corrupt, twisted in favor of the wealthy and powerful, and the law she dedicated her years to studying seems non-existent.

Full text below: 
Harper Davis couldn't believe her eyes. The idyllic town where she had grown up, where she had learned to ride a bike and found her first love, no longer seemed to exist. In its place, she found an unrecognizable landscape plagued by injustice and corruption. 

After graduating from an ivy league law school, Harper was rea

In [ ]:
print("".join(drafts))

Harper Davis couldn't believe her eyes. The idyllic town where she had grown up, where she had learned to ride a bike and found her first love, no longer seemed to exist. In its place, she found an unrecognizable landscape plagued by injustice and corruption. 

After graduating from an ivy league law school, Harper was ready to dive into the murky waters of her profession. It was her dream. Her goal. Law was the means by which she intended to provoke change. But upon returning to her hometown, that dream quickly turned into a nightmare.

The justice system, which Harper held sacred, bore little resemblance to the one she had eagerly studied in law school. Instead of truth and equality, it leaned toward favoring the wealthy, shunning the needy, and bending to the will of those in power.

Among these powerful figures was the charismatic, seemingly affable, councilman, Donovan "Donny" Grant. He once coached Harper's little league softball team, doling out joyful smiles and inspiring speec

## Revision

In [ ]:
# Progressive extraction
facts = []
edits = []
for idx in range(1, len(drafts)):
    # Old Facts
    facts_prompt = f"What facts can be inferred from this text?\n\nText:\n{drafts[idx-1]}\n\nFacts:\n-"
    facts_response = complete(facts_prompt)
    facts_response = facts_response.strip()
    facts.append(facts_response)

    # New Facts
    facts_prompt = f"What facts can be inferred from this text?\n\nText:\n{drafts[idx]}\n\nFacts:\n-"
    facts_response = complete(facts_prompt)
    facts_response = facts_response.strip()

    # Consistency
    joined_facts = "\n".join(facts)
    consistency_prompt = f"Are any New Facts inconsistent with Old Facts?:\n\Old facts:\n{joined_facts}\n\nNew facts:{facts_response}\n\nInconsistencies:\n-"
    consistency_response = complete(consistency_prompt)

    # Rewriting
    edit_prompt = f"Edit so that any inconsistencies are addressed:\nInconsistencies:{consistency_response}\n\nText:{drafts[idx]}\n\nRewritten Text:"
    edit_response = complete(edit_prompt)
    edit_response = edit_response.strip()
    edits.append(edit_response)

    print(f"Draft Text {idx+1}\n-----\n", drafts[idx], f"\n\nOld Facts:\n{joined_facts}", f"\n\nNew Facts:\n{facts_response}", f"\n\nConsistency:\n{consistency_response}", f"\n\nRewritten Text:\n{edit_response}\n-----\n\n")


Draft Text 2
-----
 The day Harper ran into councilman Donovan Grant was an unexpected encounter. He was stepping down from the stage at a town celebration, his electric charisma still warming the applause of the crowd. He spotted her in the corner, the "prodigal daughter," as he enthusiastically referred to her.

"Well, if it isn't the famous Harper Davis," he drawled, his voice dripping with exaggerated affection. "Back in town after mixing it up with the big city folks, eh? Law school and all that?” He winked at her, the twinkle in his eyes as disarming as ever.

“Right.” Harper forced a thin smile, matching his overly sweet tone. “So, how are things here, councilman Grant?”

“Ah, just as they always were, Harper, just as they always were. We try to keep things running smoothly around here.”

Underneath his charming facade, Harper saw something else. A slickness, a cunning that was too shrewd for her liking. His words seem to echo those she had been hearing all along - flirting with

In [ ]:
"/n".join(edits)

'The unexpected meeting of Harper with councilman Donovan Grant happened on the day of a town celebration, right as he was stepping down from the stage. His charisma, as electric as ever, had warmed the applause of the crowd which was still resonating in the air. In a corner, he spotted Harper, or as he was fond of calling her, the "prodigal daughter". \n\n"Well, look what the cat dragged in—our renowned Harper Davis," he exclaimed, his voice weighty with exaggerated affection. "Back from rubbing shoulders with the high-profile city folks, and a law school grad to boot?” His wink was just as disarming as his words. \n\n“Indeed,” Harper said, her smile thin and forced, mirroring his affected tone. “How have things been around here, Councilman Grant?”\n\n“Oh, everything\'s on an even keel, Harper; business as usual. It\'s all about maintaining the status quo.”\n\nBehind his disarming charm, Harper detected hints of a shrewd cunning that was unpalatable. His sugarcoated words seemed to mi